### Importación librerías para limpieza y transformación de datos.

In [3]:
import pandas as pd
import glob
import re

from datetime import datetime

import numpy as np 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

### 1. Limpieza tabla Padrón Madrid

#### 1.1 Formato long

- Nos hemos propuesto obtener todos los .csv de la carpeta, escritos con una estructura bastante exhaustiva. Nuestra tarea será agruparlos por distrito y sumar la cantidad total de nacionales y extranjeros empadronados por mes (cada .csv contiene un mes)

- Vamos a almacenar todos esos dataframes y a iterar sobre ellos, buscando la fecha en el nombre de archivo, ya que todos tienen una estructura tipo 'Rango_Edades_Seccion_*YYYYMM*.csv, para crear una variable con la fecha.

- Luego, vamos a sumar las columnas de españoles y extranjeros para ver cuanta gente vive en cada barrio en cada momento.
Hemos tenido que usar un 'contains' y que, a veces, estaba escrito como españoles y otras como espanoles.

- Finalmente nos quedamos con el distrito, los nacionales y los extranjeros y añadimos una columna total, agrupando por distrito y añadiendo el distrito 'Madrid' que contenga la suma de todos. Concatenamos esa fila a cada uno de los archivos por mes que tenemos, y le incorporamos el mes en el que ha ocurrido, formateándolo con datetime para que tenga una estructura de DD/MM/YY.

- Incorporamos cada uno de estos dataframes a nuestra lista de dataframes y los concatenamos para acabar exportando un archivo .csv del histórico del padrón agrupado por distrito.

In [11]:


# Obtenemos la lista de .csv y creamos una lista para ir almacenando los dataframes
archivos_csv = glob.glob('D:/IMMUNE/9. CAPSTONE PROJECT/PADRON/Rango_Edades_Seccion_*.csv')
dataframes = []

# Ahora vamos a iterar sobre los archivos
for archivo in archivos_csv:
    # Obtenemos la fecha del nombre del arhivo, y leemos el .csv
    fecha = re.search(r'Rango_Edades_Seccion_(\d+)', archivo).group(1)
    data = pd.read_csv(archivo, encoding='latin-1', delimiter=';')
    
    # Vamos a obtener los nombres de las columnas (cada una ha sido escrita de diferente manera, respetando o no la 'ñ', y también vamos a no distinguir entre mayusculas y minusculas)
    columnas_espanoles_hombres = data.columns[data.columns.str.contains('espanoles|españoles', case=False)].tolist()
    columnas_espanoles_mujeres = data.columns[data.columns.str.contains('espanolas|españolas', case=False)].tolist()
    columnas_extranjeros_hombres = data.columns[data.columns.str.contains('extranjeros', case=False)].tolist()
    columnas_extranjeros_mujeres = data.columns[data.columns.str.contains('extranjeras', case=False)].tolist()
    
    # Sumamos los valores para obtener las columnas de 'Nacionales' y 'Extranjeros', y luego nos quedamos solo con las relevantes.
    # Manipulamos los nombres de las columnas para que sigan el estándar que hemos definido con nuestros compañeros

    data['Nacionales'] = data[columnas_espanoles_hombres].sum(axis=1) + data[columnas_espanoles_mujeres].sum(axis=1)
    data['Extranjeros'] = data[columnas_extranjeros_hombres].sum(axis=1) + data[columnas_extranjeros_mujeres].sum(axis=1)
    ag_data = data[['Nacionales', 'Extranjeros', 'DESC_DISTRITO']]
    final = ag_data.copy()
    final['DESC_DISTRITO'] = final['DESC_DISTRITO'].replace({
        '.*FUENCARRAL-EL PARDO.*': 'FUENCARRAL',
        '.*MONCLOA.*': 'MONCLOA',
        '.*PUENTE DE VALLECAS.*': 'PUENTE_VALLECAS',
        '.*SAN BLAS.*': 'SAN:BLAS',
        '.*VILLA DE VALLECAS.*': 'VILLA_VALLECAS',
        '.*CIUDAD LINEAL.*': 'CIUDAD_LINEAL'
    }, regex=True)
    
    # Agrupamos, sumamos por distrito, calculasmos los totales, creamos la fila 'madrid' y la agregamos al dataframe
    agrupado = final.groupby('DESC_DISTRITO').sum()
    
    total_nacionales = agrupado['Nacionales'].sum()
    total_extranjeros = agrupado['Extranjeros'].sum()
    
    madrid_row = pd.DataFrame([[total_nacionales, total_extranjeros]], columns=['Nacionales', 'Extranjeros'], index=['MADRID'])
    agrupado = pd.concat([agrupado, madrid_row])
    
    # Convertimos las columnas a 'int', y formateamos fecha para que acabe con el formato que buscávamos, añadiéndolo al dataframe
    agrupado['nacionales'] = agrupado['Nacionales'].astype(int)
    agrupado['extranjeros'] = agrupado['Extranjeros'].astype(int)
    agrupado['total'] = agrupado['Nacionales'] + agrupado['Extranjeros']
    
    fecha_dt = datetime.strptime(fecha, "%Y%m")
    fecha_formateada = fecha_dt.strftime("%d/%m/%y")
    agrupado['date'] = fecha_formateada

    #Renombramos el eje a 'distrito' (ya que al estar agrupado por distrito este se situó en el eje)
    agrupado = agrupado.rename_axis('distrito')
    agrupado.index = agrupado.index.str.lower()
    agrupado = agrupado.reset_index()

    # Agregamos
    dataframes.append(agrupado)

# Concatenamos y exportamos :)
resultado = pd.concat(dataframes)

resultado.to_csv('PADRON1222_LONG.csv', index=True)

- Para manipular nuestro archivo, lo primero será unificar los distritos para que todos estén en minúscula y sin espacios que puedan confundir las categorías.

In [13]:
data= pd.read_csv('PADRON1222_LONG.csv')
data['distrito'] = data['distrito'].str.strip().str.lower()
data['distrito'].value_counts()


distrito
arganzuela              108
barajas                 108
madrid                  108
villa_vallecas          108
villaverde              108
vicalvaro               108
usera                   108
tetuan                  108
san:blas                108
salamanca               108
retiro                  108
puente_vallecas         108
moratalaz               108
moncloa                 108
latina                  108
hortaleza               108
fuencarral              108
chamberi                108
chamartin               108
centro                  108
carabanchel             108
ciudad_lineal           107
dto. fict.sec.desap.      4
dtosec.inex. en bdc       1
ciudad lienal             1
Name: count, dtype: int64

- Revisando el archivo obtenido, vemos que hay unos valores como 'dto. fict.sec.desap.' 'dtosec.inex. en bdc'. 

- Al comprobarlos en los datos originales, comprobamos que no sólo se trata de registros insignificantes de apenas 20 personas, sino que eran erróneos en los archivos obtenidos. Concluimos que fue un error de recogida de datos y decidimos borrarlos.

- También hay un error en el que se registró ciudad lineal como 'ciudad lienal', y decidimos enmendarlo también junto con la nomenclatura de san:blas, que debería ser san_blas.

- Se añade columna month y year.

In [15]:
data['distrito'] = data['distrito'].replace({'ciudad lienal':'ciudad_lineal',
                                             'san:blas':'san_blas'})
data = data.drop(data[(data['distrito'] == 'dto. fict.sec.desap.') | (data['distrito'] == 'dtosec.inex. en bdc')].index)

data = data[['distrito', 'nacionales', 'extranjeros', 'total', 'date']]
data['date'] = pd.to_datetime(data['date'], format='%d/%m/%y')
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year
data.head()


,distrito,nacionales,extranjeros,total,date,month,year
0,arganzuela,135269,16783,152052.0,2014-01-01,1,2014
1,barajas,41213,4508,45721.0,2014-01-01,1,2014
2,carabanchel,201419,42154,243573.0,2014-01-01,1,2014
3,centro,104359,30578,134937.0,2014-01-01,1,2014
4,chamartin,129404,13720,143124.0,2014-01-01,1,2014


In [16]:
data.to_csv('padron_12_22.csv', index=False, header=True)

#### 1.2 Formato 'wide'

- Para pasar esta tabla tipo 'long' (muchas filas, pocas columnas) a un formato 'wide' que podría sernos útil a la hora de realizar determinados cálculos, decidimos trabajar con el .csv que ya acabamos de crear. 

- Al importarlo hemos tenido que asegurarnos de nuevo de que nuestra columna 'date' y nuestras columnas de empadronados estén en el formato adecuado (datetime para 'date' e int para las demás).

- Para pasarla a 'wide' hemos hecho un pivot manteniendo el eje en el mes, y basando las columnas en las diferentes etiquetas de 'distrito', renombrándolas luego (usando la jerarquía resultante de ese pivot) para que el nombre del distrito aparezca primero, seguido de una barra baja '_' y el concepto al que hace referencia la columna.

- Posteriormente hemos tenido que organizar las columnas alfabéticamente con un reindex, y ordenar el índice por meses y no por años.

In [21]:

data = pd.read_csv('padron_12_22_long.csv')


data[['nacionales', 'extranjeros', 'total']] = data[['nacionales', 'extranjeros', 'total']].astype(int)


data_wide = data.pivot(index=['date', 'month', 'year'], columns='distrito', values=['nacionales', 'extranjeros', 'total'])

data_wide.columns = [f'{col[1]}_{col[0]}' for col in data_wide.columns]

data_wide = data_wide.reindex(sorted(data_wide.columns), axis=1)

data_wide = data_wide.sort_index()


In [22]:
data_wide.to_csv('padron_12_22_wide.csv')

### 2. Limpieza tabla Alquiler Idealista

#### 2.1 Formato long

- Aunque la tabla extraída de idealista ya estaba bastante depurada, aún hay algunos cambios por realizar.

- El primero es renombrar la columna 'zona' a 'distrito', para respetar la nomenclatura elegida.

In [23]:
alquiler = pd.read_csv('ALQUILER1223LONG.csv', encoding='latin-1')
alquiler.rename(columns={'zona': 'distrito'}, inplace=True)

In [24]:
alquiler.head()

,mes,precio_m2,var_mensual,var_trimest,var_anual,distrito
0,01/06/23,"16,8 /m2","1,20 %","3,70 %","10,20 %",GLOBAL
1,01/05/23,"16,6 /m2","1,20 %","2,60 %","11,00 %",GLOBAL
2,01/04/23,"16,4 /m2","1,20 %","2,50 %","10,30 %",GLOBAL
3,01/03/23,"16,2 /m2","0,20 %","2,70 %","10,40 %",GLOBAL
4,01/02/23,"16,2 /m2","1,10 %","2,50 %","11,20 %",GLOBAL


- Comprobamos los nombres de los distritos y vemos que hay algunos que hay que reemplazar para respetar la nomenclatura acordada. Por ejemplo, 'global' pasará a llamarse 'madrid', o 'villa de vallecas', 'villa_vallecas'.

In [25]:
alquiler['distrito'] = alquiler['distrito'].str.lower()
alquiler.distrito.value_counts()

distrito
global               136
arganzuela           136
villa de vallecas    136
vicalvaro            136
usera                136
tetuan               136
san blas             136
salamanca            136
retiro               136
puente vallecas      136
moratalaz            136
moncloa              136
latina               136
hortaleza            136
fuencarral           136
ciudad lineal        136
chamberi             136
chamartin            136
centro               136
carabanchel          136
barajas              136
villaverde           136
Name: count, dtype: int64

In [26]:
#Reemplazamos los nombres
alquiler['distrito'] = alquiler['distrito'].replace({'global': 'madrid', 
                                                         'villa de vallecas':'villa_vallecas',
                                                         'ciudad lineal':'ciudad_lineal',
                                                         'puente vallecas':'puente_vallecas',
                                                         'san blas':'san_blas'})

- Renombramos la columna 'mes' a 'date para continuar con la nomenclatura común.

In [27]:
alquiler.rename(columns={'mes': 'date'}, inplace=True)


In [30]:

alquiler['date'] = pd.to_datetime(alquiler['date'], format='%d/%m/%y')
alquiler['month'] = alquiler['date'].dt.month
alquiler['year'] = alquiler['date'].dt.year
alquiler.head()

,date,precio_m2,var_mensual,var_trimest,var_anual,distrito,month,year
0,2023-06-01,"16,8 /m2","1,20 %","3,70 %","10,20 %",madrid,6,2023
1,2023-05-01,"16,6 /m2","1,20 %","2,60 %","11,00 %",madrid,5,2023
2,2023-04-01,"16,4 /m2","1,20 %","2,50 %","10,30 %",madrid,4,2023
3,2023-03-01,"16,2 /m2","0,20 %","2,70 %","10,40 %",madrid,3,2023
4,2023-02-01,"16,2 /m2","1,10 %","2,50 %","11,20 %",madrid,2,2023


- Por último, hemos de cambiar las columnas con el formato "1,25 %" para que sean floats que contengan solo el número en cuestión. Para ello primero creamos una lista con los columnas en las que están los valores a cambiar y aplicamos un bucle for para realizar esta transformación.

In [31]:
columnas_cambiar = ['precio_m2','var_mensual','var_trimest', 'var_anual']
for col in columnas_cambiar:
    alquiler[col] = alquiler[col].str.replace(',', '.').str.extract(r'(\d+\.\d+|\d+)')
    alquiler[col] = alquiler[col].astype(float)




In [32]:
alquiler.to_csv('alquiler_12_23.csv', index=False)

#### 2.2 Formato wide

- En cuanto a la tabla Wide, los cambios realizados han sido muy parecidos a los de la tabla 'long':

- Cambio de valores numéricos por float.
- Cambio de nombres de columnas a minúscula, cambio de nombre 'global' por 'madrid'
- Añadido de columna 'month' y 'year'

In [88]:
alquiler_w = pd.read_csv('ALQUILER1223WIDE.csv', encoding='latin-1')

In [89]:
alquiler_w.head()

,Mes,GLOBAL_precio_m2,GLOBAL_var_mensual,GLOBAL_var_trimest,GLOBAL_var_anual,ARGANZUELA_precio_m2,ARGANZUELA_var_mensual,ARGANZUELA_var_trimest,ARGANZUELA_var_anual,BARAJAS_precio_m2,...,VICALVARO_var_trimest,VICALVARO_var_anual,VILLA_VALLECAS_precio_m2,VILLA_VALLECAS_var_mensual,VILLA_VALLECAS_var_trimest,VILLA_VALLECAS_var_anual,VILLAVERDE_precio_m2,VILLAVERDE_var_mensual,VILLAVERDE_var_trimest,VILLAVERDE_var_anual
0,01/06/23,"16,8 /m2","1,20 %","3,70 %","10,20 %","16,4 /m2","0,50 %","0,80 %","8,80 %","13,2 /m2",...,"1,80 %","4,00 %","12,5 /m2","-1,30 %","2,20 %","7,00 %","12,3 /m2","0,30 %","4,00 %","9,80 %"
1,01/05/23,"16,6 /m2","1,20 %","2,60 %","11,00 %","16,4 /m2","0,70 %","1,50 %","8,70 %","13,0 /m2",...,"1,10 %","5,80 %","12,6 /m2","-0,90 %","7,00 %","9,40 %","12,3 /m2","2,60 %","3,60 %","10,80 %"
2,01/04/23,"16,4 /m2","1,20 %","2,50 %","10,30 %","16,3 /m2","-0,30 %","0,30 %","7,70 %","12,6 /m2",...,"0,00 %","6,90 %","12,8 /m2","4,50 %","7,40 %","12,60 %","12,0 /m2","1,00 %","2,10 %","9,40 %"
3,01/03/23,"16,2 /m2","0,20 %","2,70 %","10,40 %","16,3 /m2","1,10 %","1,60 %","8,70 %","12,7 /m2",...,"0,10 %","6,70 %","12,2 /m2","3,40 %","2,80 %","9,60 %","11,9 /m2","-0,10 %","0,60 %","10,20 %"
4,01/02/23,"16,2 /m2","1,10 %","2,50 %","11,20 %","16,1 /m2","-0,40 %","0,80 %","7,50 %","12,5 /m2",...,"-0,50 %","7,90 %","11,8 /m2","-0,60 %","0,30 %","5,10 %","11,9 /m2","1,10 %","0,90 %","10,40 %"


In [90]:
alquiler_w.rename(columns={'Mes': 'date'}, inplace=True)

In [91]:

alquiler_w['date'] = pd.to_datetime(alquiler_w['date'], format='%d/%m/%y')
alquiler_w['month'] = alquiler_w['date'].dt.month
alquiler_w['year'] = alquiler_w['date'].dt.year
alquiler_w.head()

,date,GLOBAL_precio_m2,GLOBAL_var_mensual,GLOBAL_var_trimest,GLOBAL_var_anual,ARGANZUELA_precio_m2,ARGANZUELA_var_mensual,ARGANZUELA_var_trimest,ARGANZUELA_var_anual,BARAJAS_precio_m2,...,VILLA_VALLECAS_precio_m2,VILLA_VALLECAS_var_mensual,VILLA_VALLECAS_var_trimest,VILLA_VALLECAS_var_anual,VILLAVERDE_precio_m2,VILLAVERDE_var_mensual,VILLAVERDE_var_trimest,VILLAVERDE_var_anual,month,year
0,2023-06-01,"16,8 /m2","1,20 %","3,70 %","10,20 %","16,4 /m2","0,50 %","0,80 %","8,80 %","13,2 /m2",...,"12,5 /m2","-1,30 %","2,20 %","7,00 %","12,3 /m2","0,30 %","4,00 %","9,80 %",6,2023
1,2023-05-01,"16,6 /m2","1,20 %","2,60 %","11,00 %","16,4 /m2","0,70 %","1,50 %","8,70 %","13,0 /m2",...,"12,6 /m2","-0,90 %","7,00 %","9,40 %","12,3 /m2","2,60 %","3,60 %","10,80 %",5,2023
2,2023-04-01,"16,4 /m2","1,20 %","2,50 %","10,30 %","16,3 /m2","-0,30 %","0,30 %","7,70 %","12,6 /m2",...,"12,8 /m2","4,50 %","7,40 %","12,60 %","12,0 /m2","1,00 %","2,10 %","9,40 %",4,2023
3,2023-03-01,"16,2 /m2","0,20 %","2,70 %","10,40 %","16,3 /m2","1,10 %","1,60 %","8,70 %","12,7 /m2",...,"12,2 /m2","3,40 %","2,80 %","9,60 %","11,9 /m2","-0,10 %","0,60 %","10,20 %",3,2023
4,2023-02-01,"16,2 /m2","1,10 %","2,50 %","11,20 %","16,1 /m2","-0,40 %","0,80 %","7,50 %","12,5 /m2",...,"11,8 /m2","-0,60 %","0,30 %","5,10 %","11,9 /m2","1,10 %","0,90 %","10,40 %",2,2023


In [98]:
lista_nombres = alquiler_w.columns.tolist()
nuevos_nombres = [nombre.lower().replace('global', 'madrid') for nombre in lista_nombres]
alquiler_w.columns = nuevos_nombres

In [93]:
columnas_cambiar = [col for col in nuevos_nombres if col not in ['date', 'month', 'year']]

for col in columnas_cambiar:
    # Reemplazar comas por puntos y extraer números
    alquiler_w[col] = alquiler_w[col].str.replace(',', '.').str.extract(r'(\d+\.\d+|\d+)')
    # Convertir las columnas a tipo numérico (float)
    alquiler_w[col] = alquiler_w[col].astype(float)
    

In [94]:
alquiler_w.head()

,date,madrid_precio_m2,madrid_var_mensual,madrid_var_trimest,madrid_var_anual,arganzuela_precio_m2,arganzuela_var_mensual,arganzuela_var_trimest,arganzuela_var_anual,barajas_precio_m2,...,villa_vallecas_precio_m2,villa_vallecas_var_mensual,villa_vallecas_var_trimest,villa_vallecas_var_anual,villaverde_precio_m2,villaverde_var_mensual,villaverde_var_trimest,villaverde_var_anual,month,year
0,2023-06-01,16.8,1.2,3.7,10.2,16.4,0.5,0.8,8.8,13.2,...,12.5,1.3,2.2,7.0,12.3,0.3,4.0,9.8,6,2023
1,2023-05-01,16.6,1.2,2.6,11.0,16.4,0.7,1.5,8.7,13.0,...,12.6,0.9,7.0,9.4,12.3,2.6,3.6,10.8,5,2023
2,2023-04-01,16.4,1.2,2.5,10.3,16.3,0.3,0.3,7.7,12.6,...,12.8,4.5,7.4,12.6,12.0,1.0,2.1,9.4,4,2023
3,2023-03-01,16.2,0.2,2.7,10.4,16.3,1.1,1.6,8.7,12.7,...,12.2,3.4,2.8,9.6,11.9,0.1,0.6,10.2,3,2023
4,2023-02-01,16.2,1.1,2.5,11.2,16.1,0.4,0.8,7.5,12.5,...,11.8,0.6,0.3,5.1,11.9,1.1,0.9,10.4,2,2023


In [95]:

alquiler_w.to_csv('alquiler_12_23_wide.csv', index=False)

### 3. Limpieza tabla Venta Idealista

#### 3.1 Formato long

- En este notebook se ven las tablas VENTA1223 e HistoricoVentas. El primer dataset parte de 2012, y el segundo retrocede desde 2012 hasta el año 2001, ambas tablas organizadas mes a mes.

In [103]:
#Carga de datos
df_ventas = pd.read_csv("VENTA1223LONG.csv", encoding='latin-1')
df_ventas2 = pd.read_csv("HistoricoVentasLONG.csv")

- Lo primero es cambiar el nombre de 'zona' por 'distrito', para adaptarnos a la nomenclatura.

In [104]:
df_ventas = df_ventas.rename(columns={'zona': 'distrito'})

- Ahora quitaremos símbolos y espacios en las columnas 'var_mensual', 'var_trimest', y 'var_anual'

In [105]:
columnas_cambiar = ['var_mensual','var_trimest', 'var_anual']
for col in columnas_cambiar:
    df_ventas[col] = df_ventas[col].str.replace(',', '.').str.extract(r'(\d+\.\d+|\d+)')
    df_ventas[col] = df_ventas[col].astype(float)

In [106]:
#Para la columna precio_m2, quitamos el punto y el €/m2
df_ventas["precio_m2"] = df_ventas["precio_m2"].str.replace('.', '').str.extract(r'(\d+\.\d+|\d+)')

df_ventas["precio_m2"] = df_ventas["precio_m2"].str.replace('.', '').astype(int)

In [107]:
#Unimos la tabla original con el histórico de 2001
df_ventas_final = pd.concat([df_ventas, df_ventas2], ignore_index=True)
df_ventas_final

,mes,precio_m2,var_mensual,var_trimest,var_anual,distrito
0,01/06/23,4015,0.9,2.0,5.1,GLOBAL
1,01/05/23,3979,0.1,2.0,4.9,GLOBAL
2,01/04/23,3977,1.1,2.8,5.6,GLOBAL
3,01/03/23,3935,0.9,1.4,5.2,GLOBAL
4,01/02/23,3900,0.8,0.4,4.5,GLOBAL
...,...,...,...,...,...,...
3121,01/05/01,1592,NaN,NaN,NaN,Madrid
3122,01/04/01,1592,NaN,NaN,NaN,Madrid
3123,01/03/01,1542,NaN,NaN,NaN,Madrid
3124,01/02/01,1542,NaN,NaN,NaN,Madrid


- Ahora crearemos la columna 'date y las columnas 'month' y 'year'

In [108]:
df_ventas_final = df_ventas_final.rename(columns={'mes': 'date'})

df_ventas_final['date'] = pd.to_datetime(df_ventas_final['date'])
df_ventas_final['month'] = df_ventas_final['date'].dt.month
df_ventas_final['year'] = df_ventas_final['date'].dt.year


C:\Users\ALONSO\AppData\Local\Temp\ipykernel_5596\2652780330.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_ventas_final['date'] = pd.to_datetime(df_ventas_final['date'])


- Cambiaremos GLOBAL por madrid
- Pasaremos a string la columna 'distrito', y la colocaremos en minúscula.

In [109]:
df_ventas_final['distrito'] = df_ventas_final['distrito'].replace("GLOBAL", "madrid")
df_ventas_final["distrito"] = df_ventas_final["distrito"].astype(str)
df_ventas_final['distrito'] = df_ventas_final['distrito'].str.lower()

- Unificamos los nombres de los distritos para que sean iguales a los de la nomenclatura común.

In [110]:
df_ventas_final['distrito'] = df_ventas_final['distrito'].str.replace('moncloa-aravaca', 'moncloa')
df_ventas_final['distrito'] = df_ventas_final['distrito'].str.replace('fuencarral-el pardo', 'fuencarral')
df_ventas_final['distrito'] = df_ventas_final['distrito'].str.replace("puente vallecas", "puente_vallecas")
df_ventas_final['distrito'] = df_ventas_final['distrito'].str.replace('ciudad lineal', 'ciudad_lineal')
df_ventas_final['distrito'] = df_ventas_final['distrito'].str.replace('villa de vallecas', 'villa_vallecas')
df_ventas_final['distrito'] = df_ventas_final['distrito'].str.replace('chamartín', 'chamartin')
df_ventas_final['distrito'] = df_ventas_final['distrito'].str.replace('tetuán', 'tetuan')
df_ventas_final['distrito'] = df_ventas_final['distrito'].str.replace('vicálvaro', 'vicalvaro')
df_ventas_final['distrito'] = df_ventas_final['distrito'].str.replace('san blas', 'san_blas')


- Sustituimos los NaN por 0, para facilitar la visualización en gráficos.

In [111]:
df_ventas_final = df_ventas_final.fillna(0)

In [112]:
#Exportamos la tabla limpiada
df_ventas_final.to_csv("ventas_01_23.csv", index=False)

#### 3.2 Formato wide

- En cuanto a la tabla Wide, los cambios realizados han sido muy parecidos a los de la tabla 'long':

- Cambio de valores numéricos por float.
- Cambio de nombres de columnas a minúscula, cambio de nombre 'global' por 'madrid'
- Añadido de columna 'month' y 'year'

In [84]:
venta_w = pd.read_csv('VENTA1223WIDE.csv', encoding='latin-1')

In [85]:
venta_w.head()

,Mes,GLOBAL_precio_m2,GLOBAL_var_mensual,GLOBAL_var_trimest,GLOBAL_var_anual,ARGANZUELA_precio_m2,ARGANZUELA_var_mensual,ARGANZUELA_var_trimest,ARGANZUELA_var_anual,BARAJAS_precio_m2,...,VICALVARO_var_trimest,VICALVARO_var_anual,VILLA_VALLECAS_precio_m2,VILLA_VALLECAS_var_mensual,VILLA_VALLECAS_var_trimest,VILLA_VALLECAS_var_anual,VILLAVERDE_precio_m2,VILLAVERDE_var_mensual,VILLAVERDE_var_trimest,VILLAVERDE_var_anual
0,01/06/23,4.015 /m2,"0,90 %","2,00 %","5,10 %",4.392 /m2,"-0,10 %","0,80 %","2,80 %",3.507 /m2,...,"4,70 %","3,70 %",2.640 /m2,"-1,80 %","1,40 %","3,50 %",1.926 /m2,"0,70 %","3,40 %","7,10 %"
1,01/05/23,3.979 /m2,"0,10 %","2,00 %","4,90 %",4.396 /m2,"0,50 %","1,00 %","4,80 %",3.411 /m2,...,"3,90 %","6,50 %",2.688 /m2,"1,50 %","5,40 %","7,00 %",1.912 /m2,"1,40 %","2,30 %","8,30 %"
2,01/04/23,3.977 /m2,"1,10 %","2,80 %","5,60 %",4.376 /m2,"0,40 %","1,00 %","4,40 %",3.351 /m2,...,"1,70 %","11,20 %",2.648 /m2,"1,60 %","3,70 %","6,40 %",1.885 /m2,"1,20 %","1,60 %","8,20 %"
3,01/03/23,3.935 /m2,"0,90 %","1,40 %","5,20 %",4.358 /m2,"0,10 %","0,60 %","5,40 %",3.399 /m2,...,"-0,30 %","8,50 %",2.605 /m2,"2,20 %","3,40 %","6,00 %",1.862 /m2,"-0,30 %","0,80 %","7,80 %"
4,01/02/23,3.900 /m2,"0,80 %","-0,40 %","4,50 %",4.354 /m2,"0,50 %","0,50 %","5,10 %",3.407 /m2,...,"-1,60 %","8,80 %",2.549 /m2,"-0,20 %","1,70 %","5,90 %",1.869 /m2,"0,70 %","1,40 %","9,10 %"


In [96]:
venta_w.rename(columns={'Mes': 'date'}, inplace=True)

In [97]:

venta_w['date'] = pd.to_datetime(venta_w['date'], format='%d/%m/%y')
venta_w['month'] = venta_w['date'].dt.month
venta_w['year'] = venta_w['date'].dt.year
venta_w.head()

,date,GLOBAL_precio_m2,GLOBAL_var_mensual,GLOBAL_var_trimest,GLOBAL_var_anual,ARGANZUELA_precio_m2,ARGANZUELA_var_mensual,ARGANZUELA_var_trimest,ARGANZUELA_var_anual,BARAJAS_precio_m2,...,VILLA_VALLECAS_precio_m2,VILLA_VALLECAS_var_mensual,VILLA_VALLECAS_var_trimest,VILLA_VALLECAS_var_anual,VILLAVERDE_precio_m2,VILLAVERDE_var_mensual,VILLAVERDE_var_trimest,VILLAVERDE_var_anual,month,year
0,2023-06-01,4.015 /m2,"0,90 %","2,00 %","5,10 %",4.392 /m2,"-0,10 %","0,80 %","2,80 %",3.507 /m2,...,2.640 /m2,"-1,80 %","1,40 %","3,50 %",1.926 /m2,"0,70 %","3,40 %","7,10 %",6,2023
1,2023-05-01,3.979 /m2,"0,10 %","2,00 %","4,90 %",4.396 /m2,"0,50 %","1,00 %","4,80 %",3.411 /m2,...,2.688 /m2,"1,50 %","5,40 %","7,00 %",1.912 /m2,"1,40 %","2,30 %","8,30 %",5,2023
2,2023-04-01,3.977 /m2,"1,10 %","2,80 %","5,60 %",4.376 /m2,"0,40 %","1,00 %","4,40 %",3.351 /m2,...,2.648 /m2,"1,60 %","3,70 %","6,40 %",1.885 /m2,"1,20 %","1,60 %","8,20 %",4,2023
3,2023-03-01,3.935 /m2,"0,90 %","1,40 %","5,20 %",4.358 /m2,"0,10 %","0,60 %","5,40 %",3.399 /m2,...,2.605 /m2,"2,20 %","3,40 %","6,00 %",1.862 /m2,"-0,30 %","0,80 %","7,80 %",3,2023
4,2023-02-01,3.900 /m2,"0,80 %","-0,40 %","4,50 %",4.354 /m2,"0,50 %","0,50 %","5,10 %",3.407 /m2,...,2.549 /m2,"-0,20 %","1,70 %","5,90 %",1.869 /m2,"0,70 %","1,40 %","9,10 %",2,2023


In [99]:
lista_nombres = venta_w.columns.tolist()
nuevos_nombres = [nombre.lower().replace('global', 'madrid') for nombre in lista_nombres]
venta_w.columns = nuevos_nombres

In [100]:
columnas_cambiar = [col for col in nuevos_nombres if col not in ['date', 'month', 'year']]

for col in columnas_cambiar:
    # Reemplazar comas por puntos y extraer números
    venta_w[col] = venta_w[col].str.replace(',', '.').str.extract(r'(\d+\.\d+|\d+)')
    # Convertir las columnas a tipo numérico (float)
    venta_w[col] = venta_w[col].astype(float)

In [101]:
venta_w.head()

,date,madrid_precio_m2,madrid_var_mensual,madrid_var_trimest,madrid_var_anual,arganzuela_precio_m2,arganzuela_var_mensual,arganzuela_var_trimest,arganzuela_var_anual,barajas_precio_m2,...,villa_vallecas_precio_m2,villa_vallecas_var_mensual,villa_vallecas_var_trimest,villa_vallecas_var_anual,villaverde_precio_m2,villaverde_var_mensual,villaverde_var_trimest,villaverde_var_anual,month,year
0,2023-06-01,4.015,0.9,2.0,5.1,4.392,0.1,0.8,2.8,3.507,...,2.640,1.8,1.4,3.5,1.926,0.7,3.4,7.1,6,2023
1,2023-05-01,3.979,0.1,2.0,4.9,4.396,0.5,1.0,4.8,3.411,...,2.688,1.5,5.4,7.0,1.912,1.4,2.3,8.3,5,2023
2,2023-04-01,3.977,1.1,2.8,5.6,4.376,0.4,1.0,4.4,3.351,...,2.648,1.6,3.7,6.4,1.885,1.2,1.6,8.2,4,2023
3,2023-03-01,3.935,0.9,1.4,5.2,4.358,0.1,0.6,5.4,3.399,...,2.605,2.2,3.4,6.0,1.862,0.3,0.8,7.8,3,2023
4,2023-02-01,3.900,0.8,0.4,4.5,4.354,0.5,0.5,5.1,3.407,...,2.549,0.2,1.7,5.9,1.869,0.7,1.4,9.1,2,2023


In [102]:

venta_w.to_csv('venta_12_23_wide.csv', index=False)

### 4. Limpieza tabla Renta Media

#### 4.1 Renta formato long

- Para este caso operaremos sobre un dataset que contiene la renta per cápita disponible bruta de Madrid, tanto en general como por distritos, para los años comprendidos entre 2002 y 2019.


In [4]:
# Leer el excel de la tabla extraida de otro dataset mas grande y detallado por que solo necesitamos los datos de la renta per capita por distrito
df = pd.read_excel('Renta_per_capita.xlsx', header=1) 
# La primera fila es el nombre de las columnas que son los años
df

,Unnamed: 0,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018 (p),2019 (a)
0,Madrid,15044,15916,16827,17439,18772,19838,21152,20639,20302,20784,19838,19973,20205,21176,21888,22364,22542,23463
1,01. Centro,16719,17124,18035,18630,20297,21465,22492,21964,21539,22393,21797,22003,22688,23866,25092,26092,26579,27236
2,"02, Arganzuela",15961,16705,17796,18426,19863,21035,21884,21605,21705,22281,21742,22116,22479,23904,24754,25643,26295,27672
3,03. Retiro,18865,20090,20884,21088,22619,23992,26251,25694,25265,26286,24570,24553,24685,26408,27630,28753,29043,30721
4,04. Salamanca,18930,20187,20983,22342,23924,26157,28278,26061,25148,26881,24040,24304,24539,26041,28017,29082,29206,31197
5,05. Chamartín,19576,20739,21224,22001,23419,25522,27638,25472,25208,26568,23652,24514,24736,26398,27960,29062,29168,31295
6,06. Tetuán,15444,16203,17022,17709,18746,19558,21347,20276,20205,20960,19679,19831,19664,20656,21447,21731,21882,22925
7,07. Chamberí,19242,20110,20751,21525,23280,24644,26788,25403,24877,25916,23884,24639,24942,26130,27852,28894,28987,30468
8,08. Fuencarral-El Pardo,15129,16139,16957,17886,19447,20817,22727,22039,21771,22288,20866,20841,21434,22426,22883,23545,23545,24644
9,09. Moncloa-Aravaca,18209,19109,19896,20212,21805,23600,25823,23859,23759,24896,22782,23485,24137,24821,25830,26984,26786,28275


- Renombramos la primera columna como 'distrito'
- La cambiaremos a minúscula y quitaremos el número que precede a los distritos.

In [5]:
df = df.rename(columns={'Unnamed: 0': 'distrito'})



In [6]:
df['distrito'].iloc[1:] = df['distrito'].iloc[1:].str.slice(4).str.lower()


C:\Users\ALONSO\AppData\Local\Temp\ipykernel_21912\559253341.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['distrito'].iloc[1:] = df['distrito'].iloc[1:].str.slice(4).str.lower()


In [7]:
df['distrito'] = df['distrito'].str.lower()
df['distrito'] = df['distrito'].str.replace('moncloa-aravaca', 'moncloa')
df['distrito'] = df['distrito'].str.replace('fuencarral-el pardo', 'fuencarral')
df['distrito'] = df['distrito'].str.replace('puente de vallecas', 'puente_vallecas')
df['distrito'] = df['distrito'].str.replace('ciudad lineal', 'ciudad_lineal')
df['distrito'] = df['distrito'].str.replace('villa de vallecas', 'villa_vallecas')
df['distrito'] = df['distrito'].str.replace('chamartín', 'chamartin')
df['distrito'] = df['distrito'].str.replace('tetuán', 'tetuan')
df['distrito'] = df['distrito'].str.replace('vicálvaro', 'vicalvaro')
df['distrito'] = df['distrito'].str.replace('san blas', 'san_blas')
df['distrito'] = df['distrito'].str.replace('chamberí', 'chamberi')
df

,distrito,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018 (p),2019 (a)
0,madrid,15044,15916,16827,17439,18772,19838,21152,20639,20302,20784,19838,19973,20205,21176,21888,22364,22542,23463
1,centro,16719,17124,18035,18630,20297,21465,22492,21964,21539,22393,21797,22003,22688,23866,25092,26092,26579,27236
2,arganzuela,15961,16705,17796,18426,19863,21035,21884,21605,21705,22281,21742,22116,22479,23904,24754,25643,26295,27672
3,retiro,18865,20090,20884,21088,22619,23992,26251,25694,25265,26286,24570,24553,24685,26408,27630,28753,29043,30721
4,salamanca,18930,20187,20983,22342,23924,26157,28278,26061,25148,26881,24040,24304,24539,26041,28017,29082,29206,31197
5,chamartin,19576,20739,21224,22001,23419,25522,27638,25472,25208,26568,23652,24514,24736,26398,27960,29062,29168,31295
6,tetuan,15444,16203,17022,17709,18746,19558,21347,20276,20205,20960,19679,19831,19664,20656,21447,21731,21882,22925
7,chamberi,19242,20110,20751,21525,23280,24644,26788,25403,24877,25916,23884,24639,24942,26130,27852,28894,28987,30468
8,fuencarral,15129,16139,16957,17886,19447,20817,22727,22039,21771,22288,20866,20841,21434,22426,22883,23545,23545,24644
9,moncloa,18209,19109,19896,20212,21805,23600,25823,23859,23759,24896,22782,23485,24137,24821,25830,26984,26786,28275


- Cambiamos los nombres de las columnas para que solo queden los años en número y los convertimos en numéricos


In [10]:
df = df.rename(columns={'2018 (p)': '2018', '2019 (a)': '2019'})
df['2018'] = pd.to_numeric(df['2018'], errors='coerce')
df['2019'] = pd.to_numeric(df['2019'], errors='coerce')
df

,distrito,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,madrid,15044,15916,16827,17439,18772,19838,21152,20639,20302,20784,19838,19973,20205,21176,21888,22364,22542,23463
1,centro,16719,17124,18035,18630,20297,21465,22492,21964,21539,22393,21797,22003,22688,23866,25092,26092,26579,27236
2,arganzuela,15961,16705,17796,18426,19863,21035,21884,21605,21705,22281,21742,22116,22479,23904,24754,25643,26295,27672
3,retiro,18865,20090,20884,21088,22619,23992,26251,25694,25265,26286,24570,24553,24685,26408,27630,28753,29043,30721
4,salamanca,18930,20187,20983,22342,23924,26157,28278,26061,25148,26881,24040,24304,24539,26041,28017,29082,29206,31197
5,chamartin,19576,20739,21224,22001,23419,25522,27638,25472,25208,26568,23652,24514,24736,26398,27960,29062,29168,31295
6,tetuan,15444,16203,17022,17709,18746,19558,21347,20276,20205,20960,19679,19831,19664,20656,21447,21731,21882,22925
7,chamberi,19242,20110,20751,21525,23280,24644,26788,25403,24877,25916,23884,24639,24942,26130,27852,28894,28987,30468
8,fuencarral,15129,16139,16957,17886,19447,20817,22727,22039,21771,22288,20866,20841,21434,22426,22883,23545,23545,24644
9,moncloa,18209,19109,19896,20212,21805,23600,25823,23859,23759,24896,22782,23485,24137,24821,25830,26984,26786,28275


- Ahora pivotamos la tabla para tener los distritos como columnas:

In [11]:
df_pivot = pd.melt(df, id_vars='distrito', var_name='Date', value_name='valor')
df_pivot = df_pivot.sort_values(by=['distrito', 'Date'])
df_pivot = df_pivot.reset_index(drop=True)

# Ver el resultado
print(df_pivot)


       distrito  Date  valor
0    arganzuela  2002  15961
1    arganzuela  2003  16705
2    arganzuela  2004  17796
3    arganzuela  2005  18426
4    arganzuela  2006  19863
..          ...   ...    ...
391  villaverde  2015  16522
392  villaverde  2016  16735
393  villaverde  2017  16566
394  villaverde  2018  16688
395  villaverde  2019  17191

[396 rows x 3 columns]


- Ahora llevaremos a cabo un análisis para prever cómo será la renta en los años siguientes y comprobar si sigue un patrón lógico con el que podamos trabajar. Vamos a hacer primero una prueba con 'madrid' y, si vemos que funciona bien, podremos aplicarla al resto de distritos para rellenar los valores faltantes (2020, 2021, 2022, 2023)

In [13]:
# Obtenemos los datos de madrid
distrito = 'madrid'
distrito_data = df[df['distrito'] == distrito].iloc[:, 1:].values.squeeze()

# Creamos una lista de años desde 2002 hasta 2019, y ambién una lista de años futuros para las estimaciones
years = range(2002, 2020)
future_years = [2020, 2021, 2022]

# Ajustamos nuestro modelo de regresión y realizamos las estimaciones para años futuros. 
# Calculamos la raíz del error cuadrático medio y el coeficiente de determinación
regression_model = LinearRegression()
X = [[year] for year in years]
y = distrito_data
regression_model.fit(X, y)
future_X = [[year] for year in future_years]
predictions = regression_model.predict(future_X)


print(f"Estimaciones de renta per cápita para el distrito {distrito}:")
for year, prediction in zip(future_years, predictions):
    print(f"Año {year}: {round(prediction, 2)}")

y_pred = regression_model.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
r2 = r2_score(y, y_pred)

print(f"Raíz del error cuadrático medio: {rmse:.2f}")
print(f"Coeficiente de determinación: {r2:.2f}")

Estimaciones de renta per cápita para el distrito madrid:
Año 2020: 23621.63
Año 2021: 24013.61
Año 2022: 24405.58
Raíz del error cuadrático medio: 950.87
Coeficiente de determinación: 0.82


In [14]:
desviacion_estandar = np.std(distrito_data)
print(f"Desviación estándar de renta per cápita para el distrito {distrito}: {desviacion_estandar:.2f}")

Desviación estándar de renta per cápita para el distrito madrid: 2244.93


- Vemos que el coeficiente de determinación sugiere que el modelo tiene una buena capacidad para explicar la variación en la renta per cápita, teniendo una raíz error cuadrático medio de menos de la mitad de la desviación estándar.

- Con estos datos, pasamos a calcular la regresión lineal para cada uno de los distritos, buscando completar los años que faltan.

In [15]:

years = range(2002, 2020)
future_years = [2020, 2021, 2022, 2023]

# Lista para almacenar los resultados de las estimaciones para cada distrito
estimations = []

# Bucle para iterar sobre cada distrito, creando un modelo de regresión lineal para los datos históricos.
#Ajustaremos el modelo para cada uno de los distritos, realizando las estimaciones en los años faltantes.
#Finalmente, se convertirán los valores predichos a int.
#Almacenaremos cada uno de estos dataframes futuros en nuestra lista "estimations"
for distrito in df_pivot['distrito'].unique():
    distrito_data = df_pivot[df_pivot['distrito'] == distrito]
    regression_model = LinearRegression()
    X = [[year] for year in years]
    y = distrito_data['valor'].values
    regression_model.fit(X, y)

    future_X = [[year] for year in future_years]
    predictions = regression_model.predict(future_X)

    estimation_df = pd.DataFrame({
        'distrito': [distrito] * len(future_years),
        'Date': future_years,
        'valor': predictions.astype(int)  # Convertir a entero
    })
    estimations.append(estimation_df)

# Concatenamos los dataframes resultantes con el dataframe original
df_estimations = pd.concat([df_pivot] + estimations, ignore_index=True)

print(df_estimations)

           distrito  Date  valor
0        arganzuela  2002  15961
1        arganzuela  2003  16705
2        arganzuela  2004  17796
3        arganzuela  2005  18426
4        arganzuela  2006  19863
..              ...   ...    ...
479  villa_vallecas  2023  22859
480      villaverde  2020  17715
481      villaverde  2021  17943
482      villaverde  2022  18172
483      villaverde  2023  18401

[484 rows x 3 columns]


- Ahora adaptaremos la columna 'Date' a minúsculas para cumplir con la nomenclatura
- Añadiremos una columna 'year' que incluya el año de cada uno de los datos.

In [16]:
df_estimations = df_estimations.rename(columns={'Date':'date'})

In [17]:

df_estimations['date'] = df_estimations['date'].astype(str)
df_estimations['year'] = df_estimations['date']
df_estimations['date'] = df_estimations['date'].apply(lambda x: '01/01/' + x)

print(df_estimations)

           distrito        date  valor  year
0        arganzuela  01/01/2002  15961  2002
1        arganzuela  01/01/2003  16705  2003
2        arganzuela  01/01/2004  17796  2004
3        arganzuela  01/01/2005  18426  2005
4        arganzuela  01/01/2006  19863  2006
..              ...         ...    ...   ...
479  villa_vallecas  01/01/2023  22859  2023
480      villaverde  01/01/2020  17715  2020
481      villaverde  01/01/2021  17943  2021
482      villaverde  01/01/2022  18172  2022
483      villaverde  01/01/2023  18401  2023

[484 rows x 4 columns]


In [18]:
df_estimations.to_csv('df_renta_per_capita_long.csv')

#### 4.2 Tabla formato wide

- En cuanto a la tabla Wide, hemos hecho un pivot sobre la tabla long ya existente



In [19]:
data = pd.read_csv('df_renta_per_capita_long.csv')

In [20]:
data.head()

,Unnamed: 0,distrito,date,valor,year
0,0,arganzuela,01/01/2002,15961,2002
1,1,arganzuela,01/01/2003,16705,2003
2,2,arganzuela,01/01/2004,17796,2004
3,3,arganzuela,01/01/2005,18426,2005
4,4,arganzuela,01/01/2006,19863,2006


In [21]:
data = data[['distrito', 'date', 'valor', 'year']]

In [26]:
wide_data = data.pivot_table(index=['date', 'year'], columns='distrito', values='valor').reset_index()


wide_data.columns.name = None  
wide_data = wide_data.rename(columns={'date': 'Fecha', 'year': 'Año'})


In [27]:
wide_data.to_csv('df_renta_per_capita_wide.csv')

### 5. Limpieza tabla Variables Sociodemográficas

- Lo primero que haremos es cargar el archivo de datos de paro, calidad de vida y sensación de seguridad en dataframes.

In [66]:
df_paro = pd.read_excel('demograficos_paro.xlsx')
df_cdv = pd.read_excel('demograficos_cdv_barrio.xlsx')
df_seguridad = pd.read_excel('demograficos_seguridad.xlsx')

#### 5.1 - Paro

In [67]:
df_paro.head()

,año,madrid,centro,arganzuela,retiro,salamanca,chamartin,tetuan,chamberi,fuencarral,...,usera,puente_vallecas,moratalaz,ciudad_lineal,hortaleza,villaverde,villa_vallecas,vicalcaro,san_blas,barajas
0,2016,197753.0,9008.0,8290.0,4788.0,5717.0,5526.0,9777.0,5743.0,10621.0,...,10840.0,21403.0,5953.0,12867.0,9178.0,12533.0,7982.0,5144.0,10595.0,2241.0
1,2017,182379.0,8457.0,7693.0,4504.0,5342.0,5141.0,5141.0,5295.0,9890.0,...,9918.0,19447.0,5642.0,11535.0,8666.0,11379.0,7684.0,4918.0,9662.0,2031.0
2,2018,167871.0,7807.0,6946.0,4200.0,4916.0,4681.0,8067.0,4895.0,8994.0,...,17950.0,26426.0,5244.0,10530.0,7966.0,10733.0,7130.0,4508.0,8770.0,1744.0
3,2019,167188.0,7595.0,6802.0,4097.0,4717.0,4577.0,8193.0,4832.0,8951.0,...,9529.0,17806.0,5018.0,10467.0,7856.0,10808.0,7306.0,4550.0,8747.0,1894.0
4,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Empecemos con 'paro', lo primero que haremos será renombrar la columna año, transformarla a datetime e incluir solo el año.

In [68]:
df_paro = df_paro.rename(columns={'año': 'date'})

In [69]:
df_paro['date'] = pd.to_datetime(df_paro['date'], format='%Y')

- Creamos una función tanto para este df como para el de calidad de vida para obtener la media entre los años 2019 y 2021 y añadirlo en la columna de 2020. Consideramos que el valor resultante de esta media es el que más se puede acercar al valor real, ya que no cambiaría significativamente en el año del confinamiento.

In [70]:
def añadir_valor_faltante(df):
    columnas_barrios = df.columns[1:]

    for columna in columnas_barrios:
        valor_2019 = df[df['date'] == '2019-01-01'][columna].values[0]
        valor_2021 = df[df['date'] == '2021-01-01'][columna].values[0]
        media = (valor_2019 + valor_2021) / 2
        df.loc[df['date'] == '2020-01-01', columna] = media

    return df

añadir_valor_faltante(df_paro)

,date,madrid,centro,arganzuela,retiro,salamanca,chamartin,tetuan,chamberi,fuencarral,...,usera,puente_vallecas,moratalaz,ciudad_lineal,hortaleza,villaverde,villa_vallecas,vicalcaro,san_blas,barajas
0,2016-01-01,197753.0,9008.0,8290.0,4788.0,5717.0,5526.0,9777.0,5743.0,10621.0,...,10840.0,21403.0,5953.0,12867.0,9178.0,12533.0,7982.0,5144.0,10595.0,2241.0
1,2017-01-01,182379.0,8457.0,7693.0,4504.0,5342.0,5141.0,5141.0,5295.0,9890.0,...,9918.0,19447.0,5642.0,11535.0,8666.0,11379.0,7684.0,4918.0,9662.0,2031.0
2,2018-01-01,167871.0,7807.0,6946.0,4200.0,4916.0,4681.0,8067.0,4895.0,8994.0,...,17950.0,26426.0,5244.0,10530.0,7966.0,10733.0,7130.0,4508.0,8770.0,1744.0
3,2019-01-01,167188.0,7595.0,6802.0,4097.0,4717.0,4577.0,8193.0,4832.0,8951.0,...,9529.0,17806.0,5018.0,10467.0,7856.0,10808.0,7306.0,4550.0,8747.0,1894.0
4,2020-01-01,192241.0,9078.5,7739.5,4561.0,5433.5,5253.0,9431.0,5466.5,10111.0,...,11220.5,20342.0,5630.5,12137.0,8972.0,12674.0,8127.5,5083.5,10110.5,2213.0
5,2021-01-01,217294.0,10562.0,8677.0,5025.0,6150.0,5929.0,10669.0,6101.0,11271.0,...,12912.0,22878.0,6243.0,13807.0,10088.0,14540.0,8949.0,5617.0,11474.0,2532.0
6,2022-01-01,160316.0,7299.0,6188.0,3800.0,4388.0,4296.0,7623.0,4318.0,8500.0,...,9239.0,17206.0,4854.0,9969.0,7710.0,10724.0,6971.0,4505.0,8593.0,1771.0


- Haremos un melt para 'girar' el dataframe

In [71]:
# Utilizar el método melt para girar el DataFrame
df_paro_girado = pd.DataFrame(df_paro.melt(id_vars='date', var_name='distrito', value_name='valor'))

In [72]:
df_paro_girado.rename(columns={'valor': 'paro'}, inplace=True)

- Guardamos el archivo limpio

In [73]:
df_paro_girado.to_csv('demograficos_paro_long.csv', index=False)

#### 5.2 - Calidad de vida

In [74]:
df_cdv.head()

,año,madrid,centro,arganzuela,retiro,salamanca,chamartin,tetuan,chamberi,fuencarral,...,usera,puente_vallecas,moratalaz,ciudad_lineal,hortaleza,villaverde,villa_vallecas,vicalcaro,san_blas,barajas
0,2016,66.4,66.9,74.0,81.0,73.1,73.1,63.4,74.8,74.2,...,53.1,52.9,70.6,64.8,68.3,58.3,65.4,64.8,62.1,70.0
1,2017,69.5,68.9,75.1,77.9,78.5,75.4,63.5,77.4,73.7,...,52.9,62.5,74.3,68.6,72.6,55.2,66.8,69.9,64.7,76.6
2,2018,69.5,69.2,73.3,76.9,80.1,81.6,66.8,75.2,70.6,...,65.7,61.2,69.2,72.2,71.3,49.0,66.6,67.8,65.2,79.0
3,2019,71.3,73.6,77.9,80.3,77.5,80.5,67.1,80.0,75.8,...,61.8,60.0,76.4,69.5,75.7,56.3,69.0,72.1,70.3,80.9
4,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Pasemos a 'calidad de vida', nuestro df_cdv
- Lo primero será cambiar el nombre de año

In [75]:
df_cdv = df_cdv.rename(columns={'año': 'date'})

- Multiplicamos por 10 los datos de 2021 para igualarlos a los demás años, medidos sobre 100 en vez de sobre 10

In [76]:
df_cdv.loc[df_cdv['date'] == 2021, df_cdv.columns[1:]] *= 10

- Transformamos la columna año en tipo datetime mantiendo solo el año para facilitar el posterior analisis

In [77]:
df_cdv['date'] = pd.to_datetime(df_cdv['date'], format='%Y')

- Utilizamos la función creada anteriormente para rellenar lo valores NaN de 2020

In [78]:
añadir_valor_faltante(df_cdv)

,date,madrid,centro,arganzuela,retiro,salamanca,chamartin,tetuan,chamberi,fuencarral,...,usera,puente_vallecas,moratalaz,ciudad_lineal,hortaleza,villaverde,villa_vallecas,vicalcaro,san_blas,barajas
0,2016-01-01,66.40,66.9,74.00,81.00,73.10,73.10,63.40,74.8,74.2,...,53.1,52.9,70.6,64.80,68.30,58.30,65.4,64.80,62.10,70.00
1,2017-01-01,69.50,68.9,75.10,77.90,78.50,75.40,63.50,77.4,73.7,...,52.9,62.5,74.3,68.60,72.60,55.20,66.8,69.90,64.70,76.60
2,2018-01-01,69.50,69.2,73.30,76.90,80.10,81.60,66.80,75.2,70.6,...,65.7,61.2,69.2,72.20,71.30,49.00,66.6,67.80,65.20,79.00
3,2019-01-01,71.30,73.6,77.90,80.30,77.50,80.50,67.10,80.0,75.8,...,61.8,60.0,76.4,69.50,75.70,56.30,69.0,72.10,70.30,80.90
4,2020-01-01,71.65,71.8,75.95,79.15,79.25,80.25,68.05,79.0,76.9,...,62.9,61.0,75.2,70.25,75.85,58.15,70.0,71.55,70.15,79.45
5,2021-01-01,72.00,70.0,74.00,78.00,81.00,80.00,69.00,78.0,78.0,...,64.0,62.0,74.0,71.00,76.00,60.00,71.0,71.00,70.00,78.00
6,2022-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Se intento predecir el valor correspondiente al año 2022 para cada distrito mediante una regresión lineal, pero las métricas no mostraron ser un modelo fiable para dicha predicción por lo que decidimos utilizar la media de los años anteriores en lugar de los obtenidos mediante la regresión. 

In [79]:
# Eliminamos la fila correspondiente al año 2022
df_cdv.dropna(subset=['date'], inplace=True)

# Calcular la media de cada columna para los años 2016 a 2021 y redondear a dos decimales
# Obtenemos el índice de la fila correspondiente al 2022
# Asignamos los valores de la media de cada columna en la fila del año 2022
numeric_columns = df_cdv.select_dtypes(include=[int, float]).columns

media_2016_2021 = df_cdv.loc[df_cdv['date'] <= '2021-01-01', numeric_columns].mean().round(2)

indice_2022 = df_cdv[df_cdv['date'] == '2022-01-01'].index[0]

df_cdv.loc[indice_2022, df_cdv.columns[1:]] = media_2016_2021

- Utilizamos el método 'melt' para girar el dataframe

In [80]:
df_cdv_girado = pd.DataFrame(df_cdv.melt(id_vars='date', var_name='distrito', value_name='valor'))

In [81]:
df_cdv_girado.rename(columns={'valor': 'cdv'}, inplace=True)

In [82]:
df_cdv_girado.to_csv('demograficos_cdv_long.csv', index=False)

#### 5.3 Seguridad

In [83]:
df_seguridad

,año,madrid,centro,arganzuela,retiro,salamanca,chamartin,tetuan,chamberi,fuencarral,...,usera,puente_vallecas,moratalaz,ciudad_lineal,hortaleza,villaverde,villa_vallecas,vicalcaro,san_blas,barajas
0,2016,73.3,74.4,72.3,77.7,75.8,81.7,74.8,78.1,82.1,...,66.3,70.0,77.9,65.8,80.4,62.5,68.1,74.4,74.4,80.4
1,2017,74.0,83.7,75.5,83.6,82.5,76.6,72.1,85.7,80.4,...,60.7,65.8,79.4,70.8,81.0,57.3,69.9,70.6,70.4,77.6
2,2018,75.4,82.3,75.3,79.7,82.2,86.2,80.2,77.3,80.6,...,68.7,68.0,72.4,74.3,80.8,60.1,71.9,68.0,71.0,77.6
3,2019,76.0,82.7,84.8,81.1,81.7,86.8,80.8,81.4,73.5,...,67.3,70.1,81.3,70.3,76.4,62.1,74.2,71.3,76.0,81.6
4,2020,76.0,82.7,84.8,81.1,81.7,86.8,80.8,81.4,73.5,...,67.3,70.1,81.3,70.3,76.4,62.1,74.2,71.3,76.0,81.6
5,2021,76.0,82.7,84.8,81.1,81.7,86.8,80.8,81.4,73.5,...,67.3,70.1,81.3,70.3,76.4,62.1,74.2,71.3,76.0,81.6
6,2022,8.2,8.5,8.4,8.7,8.9,8.8,8.1,8.9,8.7,...,7.3,7.6,8.4,8.2,8.5,7.0,8.1,8.1,8.2,8.7
7,2016,58.1,64.6,54.8,60.8,67.9,66.3,54.6,64.1,66.7,...,46.0,53.1,68.8,55.6,65.6,38.8,51.5,52.9,56.9,69.2
8,2017,58.7,73.1,61.8,67.2,71.1,63.6,50.8,72.1,67.0,...,43.0,48.3,59.7,56.0,68.1,38.1,50.7,55.4,55.3,57.2
9,2018,61.6,79.4,65.7,66.3,78.0,75.2,63.3,72.9,66.8,...,59.4,43.0,60.3,58.6,68.0,27.6,52.3,53.3,59.4,71.3


- Haremos los cambios habituales, como renombrar la columna 'año'

In [84]:
df_seguridad = df_seguridad.rename(columns={'año': 'date'})

- La primera aparición de año corresponde con la seguridad percibida en cada barrio durante el día y la segunda vez que aparece hace referencia a la seguridad percibida durante la noche, nos vamos a quedar con la media de ambas. Los años correspondientes con 2020 y 2021, que corresponden con la pandemia del COVID por lo que se asumen los datos de 2019.

- Multiplicamos por 10 los datos correspondientes a 2022 para igualarlos a los demás años que están medidos sobre 100 en vez de sobre 10

In [85]:
df_seguridad.loc[df_seguridad['date'] == 2022, df_seguridad.columns[1:]] *= 10

- Realizamos la media de cada año, y es el unico dato con el que nos quedamos

In [86]:
df_avg_seguridad = df_seguridad.groupby('date').mean()

df_avg_seguridad = df_avg_seguridad.reset_index()

- Usamos el melt para girar el dataframe

In [87]:
df_avg_seguridad_girado = pd.DataFrame(df_avg_seguridad.melt(id_vars='date', var_name='distrito', value_name='valor'))

In [88]:
df_avg_seguridad_girado.rename(columns={'valor': 'seguridad'}, inplace=True)


- Transformamos la columna año en tipo datetime mantiendo solo el año para facilitar el posterior analisis

In [89]:
df_avg_seguridad_girado['date'] = pd.to_datetime(df_avg_seguridad_girado['date'], format='%Y')


- Guardamos el archivo

In [90]:
df_avg_seguridad_girado.to_csv('demograficos_avg_seguridad_long.csv', index=False)

#### 5.4 - Tabla combinada valores sociodemográficos

In [95]:
df_demograficos = pd.merge(df_paro_girado, df_cdv_girado, on=['date', 'distrito'])
df_demograficos = pd.merge(df_demograficos, df_avg_seguridad_girado, on=['date', 'distrito'])

df_demograficos

,date,distrito,paro,cdv,seguridad
0,2016-01-01,madrid,197753.0,66.40,65.70
1,2017-01-01,madrid,182379.0,69.50,66.35
2,2018-01-01,madrid,167871.0,69.50,68.50
3,2019-01-01,madrid,167188.0,71.30,68.10
4,2020-01-01,madrid,192241.0,71.65,68.10
...,...,...,...,...,...
149,2018-01-01,barajas,1744.0,79.00,74.45
150,2019-01-01,barajas,1894.0,80.90,74.60
151,2020-01-01,barajas,2213.0,79.45,74.60
152,2021-01-01,barajas,2532.0,78.00,74.60


- Transformamos la variable 'date' a dd/mm/aa para igualarla a las demás tablas y facilitar su lectura, conexión y análisis. 
- A su vez también creamos una columna year que recoja solo el año, con el mismo propósito.

In [96]:
df_demograficos['year'] = df_demograficos['date'].dt.year
df_demograficos['date'] = pd.to_datetime(df_demograficos['date']).dt.strftime('%d/%m/%y')

- Reducimos los decimales de todas la columnas de nuestro DataFrame a 2 para evitar números excesivamente largos

In [97]:
df_demograficos = df_demograficos.round(2)

- Guardamos el archivo

In [98]:
df_demograficos.to_csv('demograficos_juntos_long.csv', index=False)

### 6. Limpieza tabla interés búsqueda Google

- Cargamos los datos

In [120]:
df_google = pd.read_csv('google_04_23.csv', header=1)


In [121]:
df_google.columns

Index(['Mes', 'alquiler madrid: (España)', 'Comprar Madrid: (España)',
       'alquiler Barcelona: (España)', 'Comprar Barcelona: (España)'],
      dtype='object')

- Renombramos las columnas para que sean más operables y comprensibles


In [122]:
df_google = df_google.rename(columns={'Mes': 'date'})
df_google = df_google.rename(columns={'alquiler madrid: (España)': 'alquiler_madrid'})
df_google = df_google.rename(columns={'Comprar Madrid: (España)': 'comprar_madrid'})
df_google = df_google.rename(columns={'alquiler Barcelona: (España)': 'alquiler_barcelona'})
df_google = df_google.rename(columns={'Comprar Barcelona: (España)': 'comprar_barcelona'})
df_google

,date,alquiler_madrid,comprar_madrid,alquiler_barcelona,comprar_barcelona
0,2004-01,65,23,46,15
1,2004-02,69,27,43,15
2,2004-03,59,18,48,9
3,2004-04,69,23,52,12
4,2004-05,68,20,55,9
...,...,...,...,...,...
230,2023-03,58,29,28,17
231,2023-04,67,34,29,18
232,2023-05,61,29,28,17
233,2023-06,63,28,29,16


- Cambiamos el formato de las fechas

In [123]:
df_google["date"] = pd.to_datetime(df_google["date"])

df_google['month'] = df_google['date'].dt.month

df_google['year'] = df_google['date'].dt.year


- Exportamos la tabla

In [124]:
df_google.to_csv("google_04_23.csv", index=False)

### 7. Creación tabla Master Distrito

- Crearemos una tabla enlace que incluya 22 etiquetas y categorice los barrios por precio de alquiler, precio de venta y renta media. Las 22 etiquetas serán los nombres únicos de los 21 distritos y una etiqueta para la ciudad de Madrid en general.
    


- El primer paso será leer los archivos y buscar la media de los valores que nos interesan, agrupados por distrito.

In [2]:
alquiler = pd.read_csv('alquiler_12_23_long.csv')
venta = pd.read_csv('ventas_12_23.csv')
renta = pd.read_csv('df_renta_per_capita_long.csv')

In [3]:
media_alquiler_m2_por_distrito = alquiler.groupby('distrito').precio_m2.mean()
media_venta_m2_por_distrito = venta.groupby('distrito').precio_m2.mean()
media_renta_por_distrito = renta.groupby('distrito').valor.mean()

- Ahora pasaremos a categorizar los barrios según el cuartil en el que se encuentren teniendo en cuenta estos valores. Realizamos la suma de los valores totales por distrito y retiramos la categoría de Madrid, ya que no habrá que asignarlo a ningún cuartil al ser la suma de todos los valores de la ciudad.

- Seguidamente definiremos una función para asignar los distritos a una categoría según la situación de su valor en relación a los demás distritos.

- Crearemos un dataframe en el que incorporaremos el nombre del distrito junto con su valor

In [4]:
suma_total_por_distrito = media_alquiler_m2_por_distrito + media_venta_m2_por_distrito + media_renta_por_distrito
suma_total_por_distrito_sin_madrid = suma_total_por_distrito.drop('madrid', errors='ignore')


def asignar_cuartil(valor):
    if valor >= suma_total_por_distrito_sin_madrid.quantile(0.75):
        return 'alto'
    elif valor >= suma_total_por_distrito_sin_madrid.quantile(0.5):
        return 'medio_alto'
    elif valor >= suma_total_por_distrito_sin_madrid.quantile(0.25):
        return 'medio_bajo'
    else:
        return 'bajo'


data = []
for distrito in suma_total_por_distrito_sin_madrid.index:
    suma_total = suma_total_por_distrito_sin_madrid[distrito]
    data.append({'distrito': distrito, 'grupo': asignar_cuartil(suma_total)})

data.append({'distrito': 'madrid', 'grupo': 'ciudad'})


df_final = pd.DataFrame(data)

df_final.sort_values('grupo')

,distrito,grupo
10,moncloa,alto
3,centro,alto
4,chamartin,alto
5,chamberi,alto
14,salamanca,alto
13,retiro,alto
18,vicalvaro,bajo
2,carabanchel,bajo
17,usera,bajo
12,puente_vallecas,bajo


Guardamos en csv nuestra tabla master de distrito

In [8]:
df_final.to_csv('master_distritos.csv')